# Lending disparities using Logistic Regression

**The story:** https://www.revealnews.org/article/for-people-of-color-banks-are-shutting-the-door-to-homeownership/

**Author:** Aaron Glantz and Emmanuel Martinez

**Topics:** Logistic regression, odds ratios

**Datasets**

* **philadelphia-mortgages.csv:** Philadelphia mortgage data for 2015
    - A subset of HMDA LAR data from [FFEIC](https://www.ffiec.gov/hmda/hmdaproducts.htm)
    - Codebook is `2015HMDACodeSheet.pdf`
    - A [guide to HMDA reporting](https://www.ffiec.gov/hmda/guide.htm)
    - I've massaged it slightly to make processing a bit easier
* **nhgis0006_ds233_20175_2017_tract.csv:**
    - Table B03002: Hispanic or Latino Origin by Race
    - 2013-2017 American Community Survey data US Census Bureau, from [NHGIS](https://data2.nhgis.org/main)
    - Codebook is `nhgis0006_ds233_20175_2017_tract_codebook.txt`
* **lending_disparities_whitepaper_180214.pdf:** the whitepaper outlining Reveal's methodology

## What's the goal?

Do banks provide mortgages at disparate rates between white applicants and people of color? We're going to look at the following variables to find out:

* Race/Ethnicity
    - Native American
    - Asian
    - Black
    - Native Hawaiian
    - Hispanic/Latino
    - Race and ethnicity were not reported
* Sex
* Whether there was a co-applicant
* Applicant’s annual income (includes co-applicant income)
* Loan amount
* Ratio between the loan amount and the applicant’s income
* Ratio between the median income of the census tract and the median income of the metro area
* Racial and ethnic breakdown by percentage for each census tract
* Regulating agency of the lending institution

# Setup

Import pandas as usual, but also import numpy. We'll need it for logarithms and exponents.

Some of our datasets have a lot of columns, so you'll also want to use `pd.set_option` to display up to 100 columns or so.

In [101]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm

# What is each row of our data?

If you aren't sure, you might need to look at either the whitepaper or the codebook. You'll need to look at them both eventually, so might as well get started now.

In [49]:
pd.set_option('display.max_columns', 150)

# Read in your data

Read in our Philadelphia mortgage data and take a peek at the first few rows.

* **Tip:** As always, census tract columns like to cause problems if they're read in as numbers. Make sure pandas reads it in as a string.

In [50]:
df = pd.read_csv("data/philadelphia-mortgages.csv")

### Check your column types

I mentioned it above, but make sure `census_tract` is an object (a string) or merging isn't going to be any fun later on.

In [51]:
df.dtypes

census_tract                   float64
county_code                      int64
state_code                       int64
applicant_sex                    int64
income                           int64
loan_amount                      int64
loan_type                        int64
property_type                    int64
occupancy                        int64
action_type                      int64
loan_purpose                     int64
agency_code                      int64
tract_to_msa_income_percent    float64
applicant_race                   int64
co_applicant_sex                 int64
dtype: object

In [52]:
df.census_tract = df.census_tract.astype(str)

In [53]:
df.head()

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex
0,101.0,101,42,3,26,5,1,1,1,4,2,5,97.09,6,5
1,264.0,101,42,2,26,40,1,1,1,4,2,5,98.27,3,5
2,281.0,101,42,2,22,20,1,1,1,5,2,5,72.28,6,5
3,158.0,101,42,2,57,36,1,1,1,5,3,5,105.87,6,5
4,358.0,101,42,1,80,34,1,1,1,1,3,5,139.62,5,2


# Engineering and cleaning up features

## Income-related columns

> When we plotted the number of applicants, how much money they made and the size of the loan, we found that it skewed to the left, meaning the majority of applicants were clustered on the lower end of the income and loan amount scales. This was especially true for applicants of color. **We took the logarithm transformation of income and loan amount to normalize the distribution of those variables and limit the effect of extreme outliers.**

A few of the columns you'll need to calculate yourselves. **Calculate these values and assign them to three new columns.**

* Applicant’s adjusted annual income (includes co-applicant income)
* Adjusted loan amount
* Ratio between the loan amount and the applicant’s income

Instead of using the raw income and loan amount, you'll want the log of both income and loan amount. Call these new columns `log_income` and `log_loan_amount`. The third column will be `loan_income_ratio`.

* **Tip:** `np.log` gives you the logarithm

In [54]:
df['log_income'] = np.log(df['income'])
df['log_loan_amount'] = np.log(df['loan_amount'])
df['income_ratio'] = df['loan_amount']  /  df['income']

In [55]:
df.head()

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,income_ratio
0,101.0,101,42,3,26,5,1,1,1,4,2,5,97.09,6,5,3.258097,1.609438,0.192308
1,264.0,101,42,2,26,40,1,1,1,4,2,5,98.27,3,5,3.258097,3.688879,1.538462
2,281.0,101,42,2,22,20,1,1,1,5,2,5,72.28,6,5,3.091042,2.995732,0.909091
3,158.0,101,42,2,57,36,1,1,1,5,3,5,105.87,6,5,4.043051,3.583519,0.631579
4,358.0,101,42,1,80,34,1,1,1,1,3,5,139.62,5,2,4.382027,3.526361,0.425000


### Co-applicants

Right now we have a column about the co-applicant's sex (see the codebook for column details). We don't want the sex, though, we're interested in whether there is a co applicant or not. Use the co-applicant's sex to **create a new column called `co_applicant` that is either 'yes', 'no', or 'unknown'.**

* **Hint:** If the co-applicant's sex was not provided or is not applicable, count it as unknown.
* **Hint:** The easiest way is to use `.replace` on the co-applicant sex column, but store the result in your new column

In [56]:
df['co_applicant'] = df['co_applicant_sex']

In [57]:
def sex(row):
    if row == 1:
        return "yes"
    elif row == 2:
        return "yes"
    elif row == 5:
        return "no"
    else:
        return'Unknown'
    

In [58]:
df['co_applicant'] = df['co_applicant'].apply(sex)

# Filter loan applicants

If you read the whitepaper - `lending_disparities_whitepaper_180214.pdf` - many filters are used to get to the target dataset for analysis.

> **Loan type**
>
> While we recognize the substantial presence of applicants of color in the FHA market, we focused on conventional home loans for several reasons.

> **Property type**
>
> Prospective borrowers submit loan applications for various types of structures: one- to four-unit properties, multifamily properties and manufactured homes. For this analysis, we focused on one- to four-unit properties.

> **Occupancy**
>
> We included only borrowers who said they planned to live in the house they were looking to buy. We did this to exclude developers or individuals who were buying property as an investment or to subsequently flip it.

> **Action Type**
>
> We wanted to look at the reasons lending institutions deny people a mortgage. After conversations with former officials at HUD, we decided to include only those applications that resulted in originations (action type 1) or denials (action type 3)

> **Income**
>
> An applicant’s income isn’t always reported in the data. In other cases, the data cuts off any incomes over \\$9.9 million and any loan amounts over \\$99.9 million, meaning there’s a value in the database, but it’s not precise. We focused only on those records where income and loan amount have an accurate estimation. This meant discarding about 1 percent of all conventional home loans in the country for 2016. [Note: I already edited this]
>
> When we plotted the number of applicants, how much money they made and the size of the loan, we found that it skewed to the left, meaning the majority of applicants were clustered on the lower end of the income and loan amount scales. This was especially true for applicants of color. We took the logarithm transformation of income and loan amount to normalize the distribution of those variables and limit the effect of extreme outliers.

> **Lien status**
>
> We included all cases in our analysis regardless of lien status.

> **Race and ethnicity**
>
> At first, we looked at race separate from ethnicity, but that approach introduced too many instances in which​ ​either the ethnicity or race was unknown. So we decided to combine race and ethnicity. Applicants who marked their ethnicity as Hispanic were grouped together as Hispanic/Latino regardless of race. Non-Hispanic applicants, as well as those who didn’t provide an ethnicity, were grouped together by race: non-Hispanic white, non-Hispanic black, etc. **[Note: This has already been taken care of]**

> **Loan purpose**
>
> We decided to look at home purchase, home improvement and refinance loans separately from each other. [Note: please look at **home purchase** loans.]

Use the text above (it's from the whitepaper) and the **2015HMDACodeSheet.pdf** code book to filter the dataset.

* **Tip:** there should be between 5-8 filters, depending on how you write them.

In [59]:
df= df[df['loan_type'] == 1]
df['loan_type'].unique()

array([1])

In [60]:
df= df[df['property_type'] == 1]
df['property_type'].unique()

array([1])

In [61]:
df= df[df['occupancy'] == 1]
df['occupancy'].unique()

array([1])

In [62]:
df = df.loc[(df['action_type'] == 1) | (df['action_type'] == 3)]
df['action_type'].unique()

array([1, 3])

In [63]:
df= df[df['loan_purpose'] == 1]
df['loan_purpose'].unique()

array([1])

When you're done filtering, save your dataframe as a "copy" with `df = df.copy()` (if it's called `df`, of course). This will prevent irritating warnings when you're trying to create new columns.

### Confirm that you have 10,107 loans with 19 columns

In [64]:
df = df.copy()

### Create a "loan denied" column

Right now the `action_type` category reflects whether the loan was granted or not, and either has a value of `1` or `3`.

Create a new column called `loan_denied`, where the value is `0` if the loan was accepted and `1` if the loan was denied. **This will be our target for the machine learning algorithm.**

* **Tip:** You should have 8,878 successful loans and 1,229 denied loans

In [65]:
df['loan_denied'] = df.action_type
df['loan_denied'] = df['loan_denied'].astype(str)
df['loan_denied'] = df['loan_denied'].str.replace("1","0")
df['loan_denied'] = df['loan_denied'].str.replace("3","1")
df['loan_denied'] = df['loan_denied'].astype(int)

# Deal with categorical variables

Let's go ahead and take a look at our categorical variables:

* Applicant sex (male, female, na)
* Applicant race
* Mortgage agency
* Co-applicant (yes, no, unknown)

Before we do anything crazy, let's use the codebook to turn them into strings.

* **Tip:** We already did this with the `co_applicant` column, you only need to do the rest
* **Tip:** Just use `.replace`

In [66]:
def applicant_race(row):
    if row == 1:
        return "American Indian or Alaska Native"
    elif row == 2:
        return "Asian"
    elif row == 3:
        return "Black or African American"
    elif row == 4:
        return "Native Hawaiian or Other Pacific Islander"
    elif row == 5:
        return "White"


In [67]:
df['applicant_race'] = df['applicant_race'].apply(applicant_race)
df['applicant_race'].unique()

array(['White', None, 'Asian', 'Black or African American',
       'American Indian or Alaska Native',
       'Native Hawaiian or Other Pacific Islander'], dtype=object)

In [68]:
def sex_appl(row):
    if row == 1:
        return "male"
    elif row == 2:
        return "female"



In [69]:
df['applicant_sex'] = df['applicant_sex'].apply(sex_appl)
df['applicant_sex'].unique()

array(['female', None, 'male'], dtype=object)

In [70]:
def agency(row):
    if row == 1:
        return "OCC"
    elif row == 2:
        return "FRS"
    elif row == 3:
        return "FDIC"
    elif row == 5:
        return "NCUA"
    elif row == 7:
        return "HUD"
    elif row == 9:
        return "CFPB"

In [71]:
df['agency_code'] = df['agency_code'].apply(agency)
df['agency_code'].unique()

array(['OCC', 'NCUA', 'FDIC', 'CFPB', 'FRS', 'HUD'], dtype=object)

In [72]:
df.head(1)

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,income_ratio,co_applicant,loan_denied
42,4019.0,45,42,female,59,112,1,1,1,1,1,OCC,133.09,White,5,4.077537,4.718499,1.898305,no,0


Double-check these columns match these values in the first three rows (and yes, you should have a lot of other columns, too).

|applicant_sex|agency_code|applicant_race|co_applicant|
|---|---|---|---|
|female|OCC|white|no|
|na|OCC|na|unknown|
|male|OCC|white|no|

## Dummy variables

Let's say we're at the end of the homework, and we have a column called `sex`, where `0` is female and `1` is male. After we've done our regression, we can look at the coefficient/odds ratio for `sex` and say something like **"being male gives you a 1.5x odds of being denied a loan."**

We can say this because we're looking at one column, and changing `sex` from `0` to `1` would turn the applicant male and give them a 1.5x chance of being denied (the odds ratio).

**But let's say we're looking at a column called `race` instead.** We could do the same `0`/`1` thing with white/minority, but what about white/black/asian? If we try to give them `0`/`1`/`2` our coefficient/odds ratio interpreation stops working, because we don't have a nice True/False dichotomy any more, it's now a *real number*.

* `0`: White
* `1`: Black
* `2`: Asian

Usually with numbers you can say "...for every increase of 1...", but we can't anymore - changing from White to Black (+1) isn't the same as changing from Black to Asian (+1). And you can't subtract Black from Asian to get White. And no, you also can't average together White and Asian to get Black. Just recognize that these aren't numbers, they're categories!

**How can we turn races off and on like we can turn the `sex` variable off and on?** A good option is to make *a `0`/`1` column for each race*. We can then flip each race off and on. These are called **dummy variables**.

In [73]:
pd.get_dummies(df.applicant_race, prefix='race').head()

,race_American Indian or Alaska Native,race_Asian,race_Black or African American,race_Native Hawaiian or Other Pacific Islander,race_White
42,0,0,0,0,1
43,0,0,0,0,0
46,0,0,0,0,1
48,0,1,0,0,0
51,0,1,0,0,0


Seems to take up a lot of space, but it works a lot better.

* The first person is white, so they have a `1` for white and a `0` for every other race
* The second person is N/A, so they have a `1` for N/A and a `0` for every other race
* The next three are white, asian, and asian, so they have a `1` under the appropriate column.

When you're looking at the regression output, each column has its own coefficient (and odds ratio). Since each race now has a column, **each race will also have its own odds ratio.** Asian would have one, Black would have one, Latino would have one - now we can look at the effect of each race separately. For example, you could then say something like "being Asian (e.g., `race_asian` going from `0` to `1`) gives you a 1.2x greater chance of being denied, and being Black gets you a 2.1x chance of being denied."

And no, you're never going to have more than one `1` in a row at the same time.

After you've created your dummy variables, there's one more step which has a real fun name: **one-hot encoding.**

### One-hot encoding

When we have two sexes - male and female - we can flip between them with one binary digit, `0` and `1`.

If we had three races - White, Asian, Black - using `pd.get_dummies` would make three columns, which makes sense on the surface. But why can we put TWO values in ONE column for sex, and it takes THREE columns for the THREE race values?

The truth is, it doesn't have to!

Instead of having three columns, we're only going to have two: **asian and black**. And if both of them are `0`? The applicant is white! This is called a **reference category**, and it means **the coefficients/odds ratios for asian and black are in reference to a white applicant.** So it isn't "being black gets you a 2.1x chance of being denied," it's *being black gets you a 2.1x chance of being denied compared to a white person*. For example:

|race_asian|race_black|person's race|
|---|---|---|
|1|0|Asian|
|0|1|Black|
|0|0|White|
|1|1|Not possible if your source is a single race column|

To create a one-hot encoded variable with a reference category, you write code like this:

In [74]:
dummies_race = pd.get_dummies(df.applicant_race, prefix='race').drop('race_White', axis=1)

> We usually use `.drop(columns=...)` to drop columns, but I'm using `axis=1` here because you should be familiar with it

### Make a one-hot encoded `sex` category with `female` as the reference category

You should end up with two columns: `sex_male` and `sex_na`.

In [75]:
dummies_sex = pd.get_dummies(df.applicant_sex, prefix='sex').drop('sex_female', axis=1)

## Using one-hot encoded columns

Since these one-hot encoded variables are standalone dataframes, we eventually need to combine them into our original dataframe.

We have four categorical variables - sex, race, co-applicant, and the loan agency - so we need you to **make four one-hot encoded variables**. Name them like this:

* `dummies_sex` - reference category of white
* `dummies_race` - reference category of female
* `dummies_co_applicant` - reference category of no
* `dummies_agency` - reference category of FDIC

Typically your reference category is the most common category, because it makes for the most interesting comparisons.

> **Tip:** if you're cutting and pasting from above, watch out for `.head()`
>
> **Tip:** After you've made them, use `.head(2)` to check the first couple rows of each to make sure they look okay

In [76]:
dummies_co_applicant = pd.get_dummies(df.co_applicant, prefix='co_applicant').drop('co_applicant_no', axis=1)

In [77]:
dummies_agency = pd.get_dummies(df.agency_code, prefix='agency').drop('agency_FDIC', axis=1)

In [78]:
df

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,income_ratio,co_applicant,loan_denied
42,4019.0,45,42,female,59,112,1,1,1,1,1,OCC,133.09,White,5,4.077537,4.718499,1.898305,no,0
43,4099.02,45,42,None,177,375,1,1,1,1,1,OCC,208.56,None,3,5.176150,5.926926,2.118644,Unknown,0
46,4102.0,45,42,male,150,381,1,1,1,1,1,OCC,215.35,White,5,5.010635,5.942799,2.540000,no,0
48,312.0,101,42,female,65,136,1,1,1,1,1,OCC,93.11,Asian,5,4.174387,4.912655,2.092308,no,0
51,4036.01,45,42,female,55,196,1,1,1,1,1,OCC,141.83,Asian,5,4.007333,5.278115,3.563636,no,0
68,135.0,101,42,male,145,313,1,1,1,1,1,NCUA,110.32,Asian,2,4.976734,5.746203,2.158621,yes,0
92,100.0,101,42,male,42,55,1,1,1,3,1,NCUA,89.49,Asian,5,3.737670,4.007333,1.309524,no,1
102,4052.0,45,42,female,46,50,1,1,1,3,1,NCUA,52.82,Black or African American,5,3.828641,3.912023,1.086957,no,1
104,4052.0,45,42,female,34,50,1,1,1,3,1,NCUA,52.82,Black or African American,5,3.526361,3.912023,1.470588,no,1
114,335.0,101,42,male,98,135,1,1,1,3,1,NCUA,89.45,Asian,5,4.584967,4.905275,1.377551,no,1


## Cleaning up our old dataframe

Take a look at your original dataframe real quick.

In [79]:
df.head(2)

,census_tract,county_code,state_code,applicant_sex,income,loan_amount,loan_type,property_type,occupancy,action_type,loan_purpose,agency_code,tract_to_msa_income_percent,applicant_race,co_applicant_sex,log_income,log_loan_amount,income_ratio,co_applicant,loan_denied
42,4019.0,45,42,female,59,112,1,1,1,1,1,OCC,133.09,White,5,4.077537,4.718499,1.898305,no,0
43,4099.02,45,42,None,177,375,1,1,1,1,1,OCC,208.56,None,3,5.176150,5.926926,2.118644,Unknown,0


In [80]:
df.shape

(10107, 20)

We don't need all of those columns! If we look at the list of columns we'll be using for the regression:

* Race/Ethnicity
* Sex
* Whether there was a co-applicant
* Applicant’s annual income (includes co-applicant income)
* Loan amount
* Ratio between the loan amount and the applicant’s income
* Ratio between the median income of the census tract and the median income of the metro area
* Racial and ethnic breakdown by percentage for each census tract
* Regulating agency of the lending institution

We can keep anything in that list, and remove everything else. For example, we can drop the variables we used to create the dummy variables, as we'll be adding the one-hot encoded versions in for the next step.

For "Racial and ethnic breakdown by percentage for each census tract" we'll need to join with some census data later, so we need to also keep census tract, county code and state code.

**Build a new dataframe with only the columns we're interested in, call it `numeric`.** We're calling it `numeric` because it's mostly numeric columns after the categorical ones have been removed.

> **Tip:** You can either use `.drop(columns=` to remove unwanted columns or `df = df[['col1', 'col2', ... 'col12']]` to only select the ones you're interseted in

In [81]:
numeric = df[["co_applicant","applicant_race", "applicant_sex", "income_ratio", "log_loan_amount","log_income", "tract_to_msa_income_percent", 
               "census_tract", "county_code", "state_code", "agency_code"]]


Confirm that `numeric` has 8 columns.

In [82]:
numeric

,co_applicant,applicant_race,applicant_sex,income_ratio,log_loan_amount,log_income,tract_to_msa_income_percent,census_tract,county_code,state_code,agency_code
42,no,White,female,1.898305,4.718499,4.077537,133.09,4019.0,45,42,OCC
43,Unknown,None,None,2.118644,5.926926,5.176150,208.56,4099.02,45,42,OCC
46,no,White,male,2.540000,5.942799,5.010635,215.35,4102.0,45,42,OCC
48,no,Asian,female,2.092308,4.912655,4.174387,93.11,312.0,101,42,OCC
51,no,Asian,female,3.563636,5.278115,4.007333,141.83,4036.01,45,42,OCC
68,yes,Asian,male,2.158621,5.746203,4.976734,110.32,135.0,101,42,NCUA
92,no,Asian,male,1.309524,4.007333,3.737670,89.49,100.0,101,42,NCUA
102,no,Black or African American,female,1.086957,3.912023,3.828641,52.82,4052.0,45,42,NCUA
104,no,Black or African American,female,1.470588,3.912023,3.526361,52.82,4052.0,45,42,NCUA
114,no,Asian,male,1.377551,4.905275,4.584967,89.45,335.0,101,42,NCUA


### Combining our features

We now have 1 dataframe of numeric features (and some merge columns), and 4 one-hot-encoded variables (each with their own dataframe). Combine all five dataframes into one large dataframe called `loan_features`.

Confirm that `loan_features` has 10,107 rows and 23 columns.

In [83]:
loan_features = pd.concat([dummies_sex, dummies_race, dummies_co_applicant,dummies_agency, numeric], axis=1)
loan_features.shape

(10107, 23)

# Census data

Now we just need the final piece to the puzzle, the census data. Read in the census data file, calling the dataframe `census`.

> **Tip:** As always, be sure to read the tract column in as a string. Interestingly, this time we _don't_ need to worry about the state or county codes in the same way.
>
> **Tip:** You're going to encounter a problem that you find every time you read in a file from the US government!

In [84]:
census = pd.read_csv("data/nhgis0007_ds215_20155_2015_tract.csv", encoding = "latin1")

## Rename some columns

If you like to keep your data extra clean, feel free to rename the columns you're interested in. If not, feel free to skip it!

> **Tip:** Make sure you're using the estimates columns, not the margin of error columns

In [85]:
census.head(1)

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,TRACTA,BLKGRPA,CONCITA,AIANHHA,RES_ONLYA,TRUSTA,AITSCEA,ANRCA,CBSAA,CSAA,METDIVA,NECTAA,CNECTAA,NECTADIVA,UAA,CDCURRA,SLDUA,SLDLA,ZCTA5A,SUBMCDA,SDELMA,SDSECA,SDUNIA,PUMA5A,BTTRA,BTBGA,NAME_E,ADK5E001,ADK5E002,ADK5E003,ADK5E004,ADK5E005,ADK5E006,ADK5E007,ADK5E008,ADK5E009,ADK5E010,ADK5E011,ADK5E012,ADK5E013,ADK5E014,ADK5E015,ADK5E016,ADK5E017,ADK5E018,ADK5E019,ADK5E020,ADK5E021,NAME_M,ADK5M001,ADK5M002,ADK5M003,ADK5M004,ADK5M005,ADK5M006,ADK5M007,ADK5M008,ADK5M009,ADK5M010,ADK5M011,ADK5M012,ADK5M013,ADK5M014,ADK5M015,ADK5M016,ADK5M017,ADK5M018,ADK5M019,ADK5M020,ADK5M021
0,G0100010020100,2011-2015,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,20100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 201, Autauga County, Alabama",1948,1931,1703,150,6,12,0,0,60,0,60,17,17,0,0,0,0,0,0,0,0,"Census Tract 201, Autauga County, Alabama",203,212,229,126,8,16,11,11,44,11,44,21,21,11,11,11,11,11,11,11,11


## Computed columns

According to Reveal's regression output, you'll want to create the following columns:

* Percent Black in tract
* Percent Hispanic/Latino in tract (I hope you know how Hispanic/Latino + census data works by now)
* Percent Asian in tract
* Percent Native American in tract
* Percent Native Hawaiian in tract

Notice that we don't include percent white - **because all of the other columns add up to percent white, we ignore it!** It's similar to a reference category.

> If we want to use buzzwords here, the technical reason we're not using percent white is called **collinearity.** We'll talk more about it on Friday.

In [86]:
census['black_perc'] = round(census['ADK5E004'] * 100/ census['ADK5E001'],2)
census['hisp_perc'] = round(census['ADK5E012'] * 100 / census['ADK5E001'],2)
census['asian_perc'] = round(census['ADK5E006'] * 100/ census['ADK5E001'],2)
census['namerican_perc'] = round(census['ADK5E005'] * 100/ census['ADK5E001'],2)
census['nahawaiian_perc'] = round(census['ADK5E007'] * 100/ census['ADK5E001'],2)

In [87]:
census.head(1)

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,TRACTA,BLKGRPA,CONCITA,AIANHHA,RES_ONLYA,TRUSTA,AITSCEA,ANRCA,CBSAA,CSAA,METDIVA,NECTAA,CNECTAA,NECTADIVA,UAA,CDCURRA,SLDUA,SLDLA,ZCTA5A,SUBMCDA,SDELMA,SDSECA,SDUNIA,PUMA5A,BTTRA,BTBGA,NAME_E,ADK5E001,ADK5E002,ADK5E003,ADK5E004,ADK5E005,ADK5E006,ADK5E007,ADK5E008,ADK5E009,ADK5E010,ADK5E011,ADK5E012,ADK5E013,ADK5E014,ADK5E015,ADK5E016,ADK5E017,ADK5E018,ADK5E019,ADK5E020,ADK5E021,NAME_M,ADK5M001,ADK5M002,ADK5M003,ADK5M004,ADK5M005,ADK5M006,ADK5M007,ADK5M008,ADK5M009,ADK5M010,ADK5M011,ADK5M012,ADK5M013,ADK5M014,ADK5M015,ADK5M016,ADK5M017,ADK5M018,ADK5M019,ADK5M020,ADK5M021,black_perc,hisp_perc,asian_perc,namerican_perc,nahawaiian_perc
0,G0100010020100,2011-2015,NaN,NaN,Alabama,1,Autauga County,1,NaN,NaN,20100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Census Tract 201, Autauga County, Alabama",1948,1931,1703,150,6,12,0,0,60,0,60,17,17,0,0,0,0,0,0,0,0,"Census Tract 201, Autauga County, Alabama",203,212,229,126,8,16,11,11,44,11,44,21,21,11,11,11,11,11,11,11,11,7.7,0.87,0.62,0.31,0.0


## Only keep what we need to join and process

We're only interested in the percentage columns that we computed. Create a new dataframe called `census_features` that is only those columns along with the one we'll need for joining with the mortgage data.

> * **Tip:** Remember we saved state, county and tract codes when working on the loan data

In [88]:
census_features = census[['STATEA', 'COUNTY', 'TRACTA', 'black_perc',
                          'hisp_perc','asian_perc','namerican_perc','nahawaiian_perc']]

In [89]:
census_features.head(1)

,STATEA,COUNTY,TRACTA,black_perc,hisp_perc,asian_perc,namerican_perc,nahawaiian_perc
0,1,Autauga County,20100,7.7,0.87,0.62,0.31,0.0


Confirm that your first few rows look something like this:
    
|STATEA|COUNTYA|TRACTA|pct_hispanic|pct_black|pct_amer_indian|pct_asian|pct_pac_islander|
|---|---|---|---|---|---|---|---|
|1|1|020100|0.872690|7.700205|0.308008|0.616016|0.000000|
|1|1|020200|0.788497|53.293135|0.000000|2.319109|0.000000|
|1|1|020300|0.000000|18.564690|0.505391|1.381402|0.269542|
|1|1|020400|10.490617|3.662672|1.560027|0.000000|0.000000|
|1|1|020500|0.743287|24.844374|0.000000|3.827929|0.000000|

Your column headers might be different but your numbers should match.

# Merge datasets

Merge `loan_features` and `census_features` into a new dataframe called `merged`.

Unfortunately something is a little different between our `loan_features` and `census_features` census tract columns. You'll need to fix it before you can merge.

## Cleaning

In [90]:

regex_pat = re.compile(r'[.].*')
loan_features['census_tract'] = loan_features['census_tract'].astype(str)
loan_features['census_tract'] = loan_features['census_tract'].str.replace(regex_pat, "")
loan_features['census_tract'] = loan_features['census_tract'].astype(int)
loan_features.dtypes
loan_features = loan_features.dropna()
census_features = census_features.dropna()

census_features['TRACTA'] = census_features.TRACTA.astype(str)
census_features['STATEA'] = census_features.STATEA.astype(str)
loan_features['county_code'] =  loan_features['county_code'].astype(str)                   
loan_features['state_code'] = loan_features['state_code'].astype(str)  
loan_features.dtypes

## Do the merge

In [91]:
#i know they are not 10107 but i couldn't figure it out

In [92]:
merged = loan_features.merge(census_features, left_on=[ "census_tract"], right_on=['TRACTA'])
merged.shape

(14444, 31)

Confirm you have 10107 rows and 31 columns in the merged dataframe.

# Our final dataframe

Drop all of the columns we merged on and save it as `train_df`.

In [93]:
train_df = merged.drop(columns= ['census_tract','county_code','state_code','STATEA','COUNTY','TRACTA'])

Confirm that `train_df` has 10107 rows and 25 columns.

In [97]:
train_df.isna().sum()

sex_male                                          0
race_American Indian or Alaska Native             0
race_Asian                                        0
race_Black or African American                    0
race_Native Hawaiian or Other Pacific Islander    0
co_applicant_Unknown                              0
co_applicant_yes                                  0
agency_CFPB                                       0
agency_FRS                                        0
agency_HUD                                        0
agency_NCUA                                       0
agency_OCC                                        0
co_applicant                                      0
applicant_race                                    0
applicant_sex                                     0
income_ratio                                      0
log_loan_amount                                   0
log_income                                        0
tract_to_msa_income_percent                       0
agency_code 

In [106]:
test_df = train_df[0:2000]

## Final cleanup

Because we can't have missing data before we run a regression, check the size of `train_df`, then drop any missing data and check the size again. **Confirm you don't lose any rows.**

In [ ]:
X_train = df_train.drop("Survived", axis=1) # this is the training dataset
y_train = df_train["Survived"] # this is the label :did someone survive? did someone not?
X_test = df_test.drop("Survived", axis=1) 
y_test = df_test["Survived"]

In [107]:
train_df

,sex_male,race_American Indian or Alaska Native,race_Asian,race_Black or African American,race_Native Hawaiian or Other Pacific Islander,co_applicant_Unknown,co_applicant_yes,agency_CFPB,agency_FRS,agency_HUD,agency_NCUA,agency_OCC,co_applicant,applicant_race,applicant_sex,income_ratio,log_loan_amount,log_income,tract_to_msa_income_percent,agency_code,black_perc,hisp_perc,asian_perc,namerican_perc,nahawaiian_perc
0,1,0,0,0,0,0,0,0,0,0,0,1,no,White,male,2.540000,5.942799,5.010635,215.35,OCC,0.00,90.63,0.00,0.22,0.00
1,1,0,0,0,0,0,0,0,0,0,0,1,no,White,male,2.540000,5.942799,5.010635,215.35,OCC,5.53,23.49,35.41,0.19,1.76
2,1,0,0,0,0,0,0,0,0,0,0,1,no,White,male,2.540000,5.942799,5.010635,215.35,OCC,0.00,72.43,3.27,2.20,0.00
3,1,0,0,0,0,0,0,0,0,0,0,1,no,White,male,2.540000,5.942799,5.010635,215.35,OCC,46.37,27.14,1.86,0.09,0.61
4,1,0,0,0,0,0,0,0,0,0,0,1,no,White,male,2.540000,5.942799,5.010635,215.35,OCC,1.17,33.65,1.37,0.00,0.00
5,1,0,0,0,0,0,0,0,0,0,0,1,no,White,male,2.540000,5.942799,5.010635,215.35,OCC,7.82,66.18,1.85,0.00,0.00
6,1,0,0,0,0,0,0,0,0,0,0,1,no,White,male,2.540000,5.942799,5.010635,215.35,OCC,21.62,2.86,4.20,0.21,0.00
7,1,0,0,0,0,0,0,0,0,0,0,1,no,White,male,2.540000,5.942799,5.010635,215.35,OCC,2.27,1.19,0.12,1.10,0.00
8,1,0,0,0,0,0,0,0,0,0,0,1,no,White,male,2.540000,5.942799,5.010635,215.35,OCC,43.97,34.63,4.57,0.00,0.00
9,1,0,0,0,0,0,0,0,0,0,0,1,no,White,male,2.540000,5.942799,5.010635,215.35,OCC,6.29,0.65,0.68,0.00,0.00


# Performing our regression

## Try with statsmodels

First try to run a linear regression with statsmodels, because even though sometimes it complains and breaks, the output just looks *so nice*. Instead of `sm.OLS` we'll use `sm.Logit`.

In [108]:
X = features.drop(columns='applicant_sex')
X = sm.add_constant(X)
y = features.applicant_sex

model = sm.Logit(y, X)
results = model.fit(method='lbfgs')
results.summary()


NameError: name 'features' is not defined

## Try again with sci-kit learn

But I'll be honest, I like sklearn a *lot lot lot* better. Using the coefficient to build a dataframe just seems so *nice*.

> **Tip:** When you build your model, use `LogisticRegression(C=1e9, solver='lbfgs', max_iter=4000)` - for if you don't increase `max_iter` (how long/hard it works) it'll complain it can't find an answer.

In [ ]:
X_train = df_train.drop("Survived", axis=1) # this is the training dataset
y_train = df_train["Survived"] # this is the label :did someone survive? did someone not?
X_test = df_test.drop("Survived", axis=1) 
y_test = df_test["Survived"]

### Getting your coefficients and odds ratios

After you run your regression **using sklearn**, you can use code like the below to print out an ordered list of features, coefficients, and odds ratios.

```python
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)
```

### Wait, what's the odds ratio again?

It's how much that variable affects the outcome **if all other variables stay the same.**

# Interpreting and thinking about the analysis

### Question 1

Our results aren't exactly the same as Reveal's, as I pulled a slightly different number of rows from the database and I'm not sure what exact dataset they used for census information. How are we feeling about this reproduction? **You might want check their 2015 results in the whitepaper.**

### Question 2

In the opening paragraph to the flagship piece, [Aaron and Emmanuel write](https://www.revealnews.org/article/for-people-of-color-banks-are-shutting-the-door-to-homeownership/):

> Fifty years after the federal Fair Housing Act banned racial discrimination in lending, African Americans and Latinos continue to be routinely denied conventional mortgage loans at rates far higher than their white counterparts.

If you look at the results, Hawaiians/Pacific Islanders (and maybe Native Americans) have an even higher odds ratio. **Why do they choose to talk about African Americans and Latinos instead?**

### Question 3

Write a sentence expressing the meaning of the **odds ratio** statistic for Black mortgage applicants. Find a line in [the Reveal piece](https://www.revealnews.org/article/for-people-of-color-banks-are-shutting-the-door-to-homeownership/) where they use the odds ratio.

### Question 4

Write a similar sentence about men.

### Question 5

Why did Aaron and Emmanuel choose to include the loan-to-income ratio statistic? **You might want to read the whitepaper.**

### Question 6

Credit score is a common reason why loans are denied. Why are credit scores not included in our analysis? **You might want to read the whitepaper.**

### Question 7

This data was just sitting out there for anyone to look at, they didn't even need to FOIA it. Why do you think this issue had not come up before Reveal's analysis?

### Question 8

As a result of this series, [a lot has happened](https://www.revealnews.org/blog/we-exposed-modern-day-redlining-in-61-cities-find-out-whats-happened-since/), although [recent changes don't look so good](https://www.revealnews.org/blog/cfpb-moves-to-limit-home-loan-data/). If you were reporting this story, what groups of people would you want to talk to in order to make sure you're getting the story right?

### Question 9

When they were consulting experts, Aaron and Emmanuel received a lot of conflicting accounts about whether they should include the "N/A" values for race (they ended up including it). If the experts disagreed about something like that, why do you think they went forward with their analysis?

### Question 10

What if we were working on this story, and our logistic regression or input dataset were flawed? What would be the repercussions?